In [4]:
cd ..

/home/is/akiyoshi-n/my-project


In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
from datetime import datetime
import torch
import torch.nn as nn
from torch import cuda
from src.my_project.dataset import load_dataset, load_text_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from src.my_project.dataset import preprocess_dataset, preprocess_multiclass_dataset
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

In [6]:
# データセットパス
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 出力パス
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs')
# パラメータの設定
MAX_LEN = 128 # 最大トークン数．本データでは最大トークン数が105なので，全て符号化できる．
# DROP_RATE = 0.4
# バッチサイズ：分割した時の一つのデータ数．一回で学習するデータ数
BATCH_SIZE = 16
# エポック数：何周学習させるか．
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# クロスバリデーションの分割数
num_folds = 5
# 乱数シード
SEED = 2023
# ラベル数
num_labels = 2

# デバイスの指定
device = "cuda:1" if cuda.is_available() else "cpu"

In [7]:
# データの読み込み
data = load_dataset(f"{DATASET_PATH}/act_classification_final.xlsx")
# 損失関数の定義
CRITERION = nn.CrossEntropyLoss()

/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
type(data['labels'])

list

In [14]:
data

{'texts': ['あたしのダイスキな姉御のパパは85歳で現役歯科医。そのパパが去年突然倒れた。胃癌。手術もして一度は仕事復帰。しかし。今日、再手術が。無事済んでＩＣＵ。頑張って！癌と闘う全ての人に安らぎを神様おねがいします☆',
  'テストしに歩いてたら１、２年生対象のアンケートしてるんですか…って言われたのだがすみません見た目若くて…と返してしまった嫌味みたい！\u3000童顔ってこと',
  '教授最高だった。何度も言うが、1996が大好きだ！生美貌の青空(と思い込んでる曲)と千のナイフ(と思い込んでる曲)に感動した。',
  '「ありがとう」「嬉しい」というような言葉は、かけられた側にもその気持ちが伝播する。感謝の心は、人間関係を円滑に保つ最大の潤滑油だろう。感謝の心は常日頃忘れないようにしたい。改めて、強くそう思う。',
  'さて、お家帰ってご飯にしよう。今夜は焼肉。昼間届いた豚肉で焼肉。備蓄してある肉が大量にあるから、明後日まで三日連続焼肉して、明明後日からは鍋にして三日ほど続け、それからまた焼肉という、肉々しい日々を過ごすことも可能……毎日、ちゃんとお家帰って食事が出来れば（ﾉ∀｀）',
  '今日は砂川→新青梅街道→R16二本木→飯能阿須→飯能駅→東飯能駅→小曾木街道→青梅市民会館前→新青梅街道→イオンモールむさし村山ミュー→砂川とクルマを動かしてきました。クルマの運転はそれはもう楽しいです！',
  'うちのおかぁちゃんは旧姓澤田なんやけど昔家に『沢田研二さんのお宅ですか？』って電話があったそうな\u3000たまたまそこに居合わせたうちのおとぉちゃんが『研二はいま外出しております』っていったからもう大変ｗ\u3000いくら京都出身やからって沢田に片っ端からかけてたんすかね？',
  'リプライ送って帰ってこないときの大体の例\u3000\u3000①自分の発言が流された\u3000②たまたま離籍した\u3000③返事しにくい\u3000④返しにくい\u3000⑤返信が思いつかなかった\u3000⑥一応ふぁぼった ヮ＜',
  'こんな時でも\u3000ＪＡＬは安全性・定時性・快適性を貫くべく\u3000現場は全力で頑張ってほしい！現場の力が今こそ問われる。それが企業の底力だ！！',
  '忘れたい恥ずかしい過去を黒歴史と言うが、今言ったりや

In [5]:
class JpSentiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import StratifiedKFold, KFold
from datasets import Dataset
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=2023) # random_stateを決めてため結果は変わらない

In [7]:
# 東北大バートversion2モデルを使用
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [8]:
# examples['text']を入力としてトークナイズする関数
def tokenize_function(examples):
    return tokenizer(examples['texts'], padding='max_length', truncation=True)

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [11]:
for fold, (train_index, eval_index) in enumerate(skf.split(data['texts'], data['labels'])):
    print(f"-----------------Fold: {fold+1}-----------------")

    # 文章分類を行うクラス：BertForSequenceClassification
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
    # モデルをGPUに配置
    model
    # 訓練データと評価データを辞書型で抽出
    train_dataset = {
        'texts': [data['texts'][i] for i in train_index],
        'labels': [data['labels'][i] for i in train_index]
    }
    eval_dataset = {
        'texts': [data['texts'][i] for i in eval_index],
        'labels': [data['labels'][i] for i in eval_index]
    }
    # データセットの作成
    train_encodings = tokenizer(train_dataset['texts'], padding=True, truncation=True, max_length=128, return_tensors='pt')
    train_labels = torch.tensor(train_dataset['labels'])
    eval_encodings = tokenizer(eval_dataset['texts'], padding=True, truncation=True, max_length=128, return_tensors='pt')
    eval_labels = torch.tensor(eval_dataset['labels'])
    train_dataset = JpSentiDataset(train_encodings, train_labels)
    eval_dataset = JpSentiDataset(eval_encodings, eval_labels)
    # 学習の設定
    # 学習を行う時間を取得
    timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    # ハイパーパラメータの設定
    trainingargs = TrainingArguments(
        # 学習を行った時間をファイル名にして保存
        output_dir = f"{output_dir}/{MODEL_NAME}{timestamp}/fold{fold+1}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=100,
        learning_rate=2e-5,
        seed=0,
        # 進捗バーを表示するかどうか
        disable_tqdm = True,
        load_best_model_at_end=True,
    )
    # トレーナーの設定
    trainer = Trainer(
        model=model,
        args=trainingargs,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    trainer.train()

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and retur

{'eval_loss': 0.5900701284408569, 'eval_accuracy': 0.6772727272727272, 'eval_f1': 0.6776148338784741, 'eval_precision': 0.6782085228687172, 'eval_recall': 0.6772727272727272, 'eval_runtime': 1.0272, 'eval_samples_per_second': 214.173, 'eval_steps_per_second': 4.868, 'epoch': 1.0}


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 0.6209838390350342, 'eval_accuracy': 0.7272727272727273, 'eval_f1': 0.7193209428544384, 'eval_precision': 0.7368277368277368, 'eval_recall': 0.7272727272727273, 'eval_runtime': 0.9704, 'eval_samples_per_second': 226.701, 'eval_steps_per_second': 5.152, 'epoch': 2.0}


...

{'eval_loss': 0.7502521276473999, 'eval_accuracy': 0.6590909090909091, 'eval_f1': 0.6589429909333213, 'eval_precision': 0.670924117205109, 'eval_recall': 0.6590909090909091, 'eval_runtime': 1.0527, 'eval_samples_per_second': 208.985, 'eval_steps_per_second': 4.75, 'epoch': 2.0}


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 1.2925001382827759, 'eval_accuracy': 0.5909090909090909, 'eval_f1': 0.5765404807958, 'eval_precision': 0.6352730087250212, 'eval_recall': 0.5909090909090909, 'eval_runtime': 1.1487, 'eval_samples_per_second': 191.52, 'eval_steps_per_second': 4.353, 'epoch': 3.0}


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 1.3403939008712769, 'eval_accuracy': 0.6818181818181818, 'eval_f1': 0.6803213587487781, 'eval_precision': 0.6806447628458497, 'eval_recall': 0.6818181818181818, 'eval_runtime': 0.9575, 'eval_samples_per_second': 229.755, 'eval_steps_per_second': 5.222, 'epoch': 4.0}
{'train_runtime': 50.0471, 'train_samples_per_second': 1758.344, 'train_steps_per_second': 37.964, 'train_loss': 0.3282520645543149, 'epoch': 4.0}


In [12]:
for fold, (train_index, eval_index) in enumerate(skf.split(data['texts'], data['labels'])):
    print(f"-----------------Fold: {fold+1}-----------------")

    # 文章分類を行うクラス：BertForSequenceClassification
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
    # モデルをGPUに配置
    model
    # 訓練データと評価データを辞書型で抽出
    train_dataset = {
        'texts': [data['texts'][i] for i in train_index],
        'labels': [data['labels'][i] for i in train_index]
    }
    eval_dataset = {
        'texts': [data['texts'][i] for i in eval_index],
        'labels': [data['labels'][i] for i in eval_index]
    }
    # データセットの作成
    train_encodings = tokenizer(train_dataset['texts'], padding=True, truncation=True, max_length=128, return_tensors='pt')
    train_labels = torch.tensor(train_dataset['labels'])
    eval_encodings = tokenizer(eval_dataset['texts'], padding=True, truncation=True, max_length=128, return_tensors='pt')
    eval_labels = torch.tensor(eval_dataset['labels'])
    train_dataset = JpSentiDataset(train_encodings, train_labels)
    eval_dataset = JpSentiDataset(eval_encodings, eval_labels)
    # 学習の設定
    # 学習を行う時間を取得
    timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    # ハイパーパラメータの設定
    trainingargs = TrainingArguments(
        # 学習を行った時間をファイル名にして保存
        output_dir = f"{output_dir}/{MODEL_NAME}{timestamp}/fold{fold+1}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=100,
        learning_rate=2e-5,
        seed=0,
        # 進捗バーを表示するかどうか
        disable_tqdm = True,
        load_best_model_at_end=True,
    )
    # トレーナーの設定
    trainer = Trainer(
        model=model,
        args=trainingargs,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    trainer.train()

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and retur

{'eval_loss': 0.6185951828956604, 'eval_accuracy': 0.6545454545454545, 'eval_f1': 0.645618749067025, 'eval_precision': 0.6559018545319916, 'eval_recall': 0.6545454545454545, 'eval_runtime': 1.0445, 'eval_samples_per_second': 210.627, 'eval_steps_per_second': 4.787, 'epoch': 1.0}


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 0.5883213877677917, 'eval_accuracy': 0.7090909090909091, 'eval_f1': 0.7072535885167464, 'eval_precision': 0.7083916083916085, 'eval_recall': 0.7090909090909091, 'eval_runtime': 0.9892, 'eval_samples_per_second': 222.411, 'eval_steps_per_second': 5.055, 'epoch': 2.0}


...

{'eval_loss': 0.6580135226249695, 'eval_accuracy': 0.6272727272727273, 'eval_f1': 0.6190425499558989, 'eval_precision': 0.6647727272727273, 'eval_recall': 0.6272727272727273, 'eval_runtime': 0.9007, 'eval_samples_per_second': 244.267, 'eval_steps_per_second': 5.552, 'epoch': 2.0}


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 0.6498874425888062, 'eval_accuracy': 0.6863636363636364, 'eval_f1': 0.6846424250070248, 'eval_precision': 0.6852449875705688, 'eval_recall': 0.6863636363636364, 'eval_runtime': 1.1462, 'eval_samples_per_second': 191.945, 'eval_steps_per_second': 4.362, 'epoch': 3.0}


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 0.7803714871406555, 'eval_accuracy': 0.6454545454545455, 'eval_f1': 0.6454545454545455, 'eval_precision': 0.6562121212121211, 'eval_recall': 0.6454545454545455, 'eval_runtime': 0.9106, 'eval_samples_per_second': 241.586, 'eval_steps_per_second': 5.491, 'epoch': 4.0}
{'train_runtime': 49.7838, 'train_samples_per_second': 1767.642, 'train_steps_per_second': 38.165, 'train_loss': 0.4794906816984478, 'epoch': 4.0}


In [13]:
import wandb 

ModuleNotFoundError: No module named 'wandb'

In [42]:
train_encodings = tokenizer(data['texts'], padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
train_labels = torch.tensor(data['labels']).to(device)
train_dataset = JpSentiDataset(train_encodings, train_labels)

In [38]:
for key, val in a.items():
    print(key, val.shape)

input_ids torch.Size([1100, 128])
token_type_ids torch.Size([1100, 128])
attention_mask torch.Size([1100, 128])


In [33]:
b = JpSentiDataset(a, data['labels'])

In [12]:
import torch

class JpSentiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [21]:
b = JpSentiDataset(a, data['labels'])

In [39]:
b[0]

/tmp/ipykernel_1653171/2963044426.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([    2, 11719,  6307,   896, 29588,  6146,   892,  1897,  2302,   896,
         20487,   897, 14245,  3099,   889, 13049, 17420,  1476,   829, 11145,
         20487,   862,  1523,  7390, 15066, 16080,   881,   829,  4370, 10101,
           829, 14594,   916,   873,   888,  1031,  2202,   897, 12224, 12337,
           829, 11258,   829, 13711,   828,  1330, 14594,   862,   829, 17773,
          3287,  6304,   889, 12332,  6215,   829,  5670, 23796,   888,    15,
          3762,   890,  5512,  6262, 11661,   896,  1104,   893,  1976, 23508,
           932,  3918,  3022,   860,   895, 17567,   873, 12343,   812,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [30]:
b.encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [19]:
len(train_dataset['texts'])

880

In [1]:
!nvidia-smi

Sun Dec 31 11:13:28 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro P6000                   On  | 00000000:1A:00.0 Off |                  Off |
| 26%   25C    P8               8W / 250W |      4MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [32]:
train_dataset

NameError: name 'train_dataset' is not defined

In [9]:
from datasets import Dataset, DatasetDict
train_data = Dataset.from_dict(train_dataset)
test_data = Dataset.from_dict(eval_dataset)

In [11]:
use_data = DatasetDict({'train': train_data, 'test': test_data})

In [12]:
use_data

DatasetDict({
    train: Dataset({
        features: ['texts', 'labels'],
        num_rows: 880
    })
    test: Dataset({
        features: ['texts', 'labels'],
        num_rows: 220
    })
})

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def compute_metrics(eval_pred):
    pred, labels = eval_pred
    pred = np.argmax(pred, axis=-1)
    
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [9]:
from transformers import AutoTokenizer
# 東北大バートversion2モデルを使用
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
from transformers import TrainingArguments, Trainer
# 学習を行う時間を取得
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
args = TrainingArguments(
    # 学習を行った時間をファイル名にして保存
    output_dir = {f"{output_dir}/{MODEL_NAME}{timestamp}"},
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    seed=0,
    # 進捗バーを表示するかどうか
    disable_tqdm = True,
    load_best_model_at_end=True,
)

In [ ]:
# 東北大バートversion2モデルを使用
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 交差検証の実行
trainer = ActClassifier(model_name = MODEL_NAME, num_labels=num_labels, tokenizer=tokenizer, criterion=CRITERION, device=device, seed=SEED)
trainer.train_model()


### データ作成のためのコード

In [7]:
from transformers import AutoTokenizer
# 東北大バートversion2モデルを使用
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [10]:
encoded_input = tokenizer('私は先生です．生徒に数学を教えています．')
print(encoded_input)

{'input_ids': [2, 3946, 897, 14385, 12461, 28, 12203, 893, 14167, 932, 13507, 888, 854, 12343, 28, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
tokenizer.decode(encoded_input["input_ids"])

'[CLS] 私 は 先生 です. 生徒 に 数学 を 教え て い ます. [SEP]'

In [18]:
batch_sentences = [
    '私は先生です．生徒に数学を教えています．',
    '私は生徒です．先生に数学を教えてもらっています．',
    '私は校長です．先生や生徒に生き方を教えています．',
]
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_inputs)

{'input_ids': tensor([[    2,  3946,   897, 14385, 12461,    28, 12203,   893, 14167,   932,
         13507,   888,   854, 12343,    28,     3,     0,     0,     0],
        [    2,  3946,   897, 12203, 12461,    28, 14385,   893, 14167,   932,
         13507,   888, 16744,   888,   854, 12343,    28,     3,     0],
        [    2,  3946,   897, 15681, 12461,    28, 14385,   918, 12203,   893,
         12269,  2706,   932, 13507,   888,   854, 12343,    28,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [2]:
import wandb
import os

In [ ]:
from datasets import Dataset, DatasetDict
train_data = Dataset.from_dict(train_dataset)
test_data = Dataset.from_dict(eval_dataset)

In [20]:
def tokenize_function(examples):
    return tokenizer(examples["texts"], padding=True, truncation=True, max_length=128)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": np.mean(predictions == labels)}

In [1]:
# 文章分類を行うクラス：BertForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="my-awesome-project"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

# 訓練データと評価データを辞書型で抽出
train_dataset = {
    'texts': [data['texts'][i] for i in range(900)],
    'labels': [data['labels'][i] for i in range(900)]
}
eval_dataset = {
    'texts': [data['texts'][i] for i in range(900, 1100)],
    'labels': [data['labels'][i] for i in range(900, 1100)]
}
# データセットの作成
train_data = Dataset.from_dict(train_dataset)
eval_data = Dataset.from_dict(eval_dataset)

train_dataset = train_data.map(tokenize_function, batched=True)
eval_dataset = eval_data.map(tokenize_function, batched=True)

# 学習を行う時間を取得
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
# ハイパーパラメータの設定
trainingargs = TrainingArguments(
    # 学習を行った時間をファイル名にして保存
    output_dir = f"{output_dir}/{MODEL_NAME}{timestamp}",
    report_to="wandb",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    learning_rate=2e-5,
    seed=0,
    # 進捗バーを表示するかどうか
    disable_tqdm = True,
    load_best_model_at_end=True,
)
# トレーナーの設定
trainer = Trainer(
    model=model,
    args=trainingargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

NameError: name 'AutoModelForSequenceClassification' is not defined